In [1]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation

import itertools

In [3]:
description_font = {'fontname':'Courier New'}

In [4]:
#defining our dataframes
df = pd.read_csv('planets.csv')

exodf = df.loc[(df['st_mass']<= 0.6) & (df['st_teff'] < 6000) & (df['pl_orbsmax'] > 0)]

stardf = df.drop_duplicates('pl_hostname')
stardf = stardf.loc[(stardf['st_mass']<= 0.6) & (stardf['st_teff'] < 6000) & (stardf['pl_orbsmax'] > 0)]

In [5]:
#cleaning and defining our data
bitwise_mask = (np.isfinite(stardf['st_dist'])) & (np.isfinite(stardf['st_mass'])) & (np.isfinite(stardf['st_teff']))
bitwise_mask_2 = (np.isfinite(exodf['st_dist'])) & (np.isfinite(exodf['st_mass'])) & (np.isfinite(exodf['st_teff']))

ra = stardf['ra'].loc[bitwise_mask]
dec = stardf['dec'].loc[bitwise_mask]
distance = stardf['st_dist'].loc[bitwise_mask] 
stellar_mass = stardf['st_mass'].loc[bitwise_mask]
teff = stardf['st_teff'].loc[bitwise_mask]
star_name = exodf['pl_hostname']

x = distance * np.cos(dec) * np.cos(ra)
y = distance * np.cos(dec) * np.sin(ra)
z = distance * np.sin(dec)

#axes limits for on_key codes
a = -300
b = 300
c = a
d = b
e = a
f = b

labels_column = star_name
labels = labels_column.values

colors = itertools.cycle(['#42f4bf', 'green', 'blue']) 

In [6]:
def zoom_fun(event):

    '''
    This code was written by Debjani Das and Laura Valdez for the #TeamExoplanets 
    internship at the American Museum of Natural History
    '''

    base_scale =1.5
    
    # get the current x and y limits
    cur_xlim = ax.get_xlim()
    cur_ylim = ax.get_ylim()
    cur_xrange = (cur_xlim[1] - cur_xlim[0])*.5
    cur_yrange = (cur_ylim[1] - cur_ylim[0])*.5
    xdata = event.xdata # get event x location
    ydata = event.ydata # get event y location
    if event.button == 'up':
        # deal with zoom in
        scale_factor = 1/base_scale
    elif event.button == 'down':
        # deal with zoom out
        scale_factor = base_scale
    else:
        # deal with something that should never happen
        scale_factor = 1
        print (event.button)
    # set new limits
    ax.set_xlim([xdata - cur_xrange*scale_factor,
                 xdata + cur_xrange*scale_factor])
    ax.set_ylim([ydata - cur_yrange*scale_factor,
                 ydata + cur_yrange*scale_factor])
    plt.draw() # force re-draw

In [7]:
def panning_fun(event): 

    '''
    This code was written by Debjani Das and Laura Valdez for the #TeamExoplanets 
    internship at the American Museum of Natural History
    '''

    if event.key == 'left': #if the key pressed is the left arrow
        global a #tells our function that we defined 'a' outside of the function
        global b #^
        a = a-50 #subtracts 50 from the value of a each time the key is pressed
        b = b-50 #^
        ax.set_xlim([a,b]) #resets the x-axis limits to new 'a' and 'b'
        plt.show() #re-shows the plot so we can see the change in axis

    if event.key == 'right': #if the key pressed is the right arrow
        a = a+50 #adds 50 from the value of a each time the key is pressed
        b = b+50 #^
        ax.set_xlim([a,b]) #resets the x-axis limits to new 'a' and 'b'
        plt.show() #re-shows the plot so we can see the change in axis

    if event.key == 'down': #if the key pressed is the down arrow
        global c #tells our function that we defined 'c' outside of the function
        global d #^
        c = c-50 #subtracts 50 from the value of a each time the key is pressed
        d = d-50 #^
        ax.set_ylim([c,d]) #resets the y-axis limits to new 'c' and 'd'
        plt.show() #re-shows the plot so we can see the change in axis

    if event.key == 'up': #if the key pressed is the up arrow
        c = c+50 #adds 50 from the value of a each time the key is pressed
        d = d+50 #^
        ax.set_ylim([c,d]) #resets the y-axis limits to new 'c' and 'd'
        plt.show() #re-shows the plot so we can see the change in axis

    if event.key == 'r': #if the key pressed is the r key
        global e #tells our function that we defined 'e' outside of the function
        global f #^
        e = e-50 #subtracts 50 from the value of a each time the key is pressed
        f = f-50 #^
        ax.set_zlim([e,f]) #resets the y-axis limits to new 'c' and 'd'
        plt.show() #re-shows the plot so we can see the change in axis

    if event.key == 't': #if the key pressed is the t key
        e = e+50 #adds 50 from the value of a each time the key is pressed
        f = f+50 #^
        ax.set_zlim([e,f]) #resets the y-axis limits to new 'c' and 'd'
        plt.show() #re-shows the plot so we can see the change in axis

    if event.key == 'b': #if the key pressed is the b key
        ax.set_xlim([-300,300]) #resets the x-axis limits 
        ax.set_ylim([-300,300]) #resets the y-axis limits 
        ax.set_zlim([-300,300]) #resets the z-axis limits 
        plt.show() #re-shows the plot so we can see the change in axis

In [24]:
def onpick(event): #this first line defines our function, and takes in an `event`
    
    ind = event.ind #here we select an `attribute` of our event, this one is the indices
    star_label= labels[ind]

        
    our_df = exodf.loc[star_name == star_label[0]]
    num_of_planets= our_df['pl_pnum'] #the number of planets per star
    orb_radius= our_df['pl_orbsmax'] #the semi-major axis radius of the planet

    orbradii= orb_radius.loc[np.isfinite(orb_radius)].sort_values()
    
    orbradii= orbradii.values #this turns a column into numbers
    #print(orbradii)
    
    if len(orbradii) > 0:
        planet_total = len(orbradii)
        fig2= plt.figure(facecolor='k')
        ax2= fig2.add_subplot(111,xlim=(-max(orbradii)-1, max(orbradii)+1), ylim=(-max(orbradii)-1, max(orbradii)+1))   
        
        for radius in orbradii:
            
            x = np.linspace(-1.0, 1.0, 100) #this adds enough points to complete the circle
            y = np.linspace(-1.0, 1.0, 100)
            X, Y = np.meshgrid(x,y)
            F = X**2 + Y**2 - radius #the rest of this equation draws the circle, 0.2 adds the radius
            ax2.contour(X,Y,F,[0], colors=next(colors))
        
        ax2.scatter(0,0, marker='*', s=50, c='Tomato')
        
        text = 'Name = {}\nDistance from Sun = {} parsec\nTotal Planets = {}\nEffective Temp = {} K\nMass = {} M_Sun'.format(
            star_name.values[0],our_df['st_dist'].iloc[0],num_of_planets.iloc[0],our_df['st_teff'].iloc[0],our_df['st_mass'].iloc[0])
        ax2.annotate(text, xy=(0.6, 0.8), xycoords='axes fraction', color='w', fontweight = 'bold', **description_font)
        
        ax2.set_facecolor('k')
        ax2.spines['right'].set_visible(False)
        ax2.spines['top'].set_visible(False)
        ax2.spines['left'].set_color('grey')
        ax2.spines['bottom'].set_color('grey')
        ax.tick_params(axis='x', colors='grey')
        ax.tick_params(axis='y', colors='grey')
            
        plt.gca().set_aspect('equal', adjustable='box')
        plt.show()

In [25]:
#Define the figure and scatter plot the data
fig = plt.figure(figsize=[10, 8]) 

ax = fig.add_subplot(111, facecolor='Black', projection='3d', xlim = (a,b), ylim = (c,d), zlim = (e,f))
ax.scatter(x, y, z, c = teff, cmap= 'plasma', s = 2*((10*stellar_mass)**2), marker = 'o', picker=5)  
ax.set_axis_off()


# fig1.canvas.mpl_connect('pick_event', onpick)

#Connect the matplotlib events

fig.canvas.mpl_connect('pick_event', onpick)

# fig = ax.get_figure() # get the figure of interest
fig.canvas.mpl_connect('key_press_event', panning_fun)
fig.canvas.mpl_connect('scroll_event',zoom_fun)

12